# Trabajo Práctico Nro. 3
## Mineria de Datos - 2024.
#### Tec. Universitaria en Inteligencia Artificial - FCEIA (UNR).

### Integrantes:
 * Pace, Bruno. Legajo: P-5295/7.
 * Sancho Almenar, Mariano. Legajo: S-5778/9.

[Link al repositorio](https://github.com/bpace1/TP3-Mineria-De-Datos)

In [49]:
# Manejo de datos
import pandas as pd
import numpy as np

# Gráficos
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt

# Modealdo
from sklearn.preprocessing import StandardScaler
#from mlxtend.plotting import plot_decision_regions
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV, PredefinedSplit, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, recall_score, f1_score, precision_score
from sklearn.tree import export_graphviz
from IPython.display import display
import graphviz
from scipy.stats import randint



# Tipado
from typing import Dict, Any, List, Generator

# Wanings
import warnings
warnings.filterwarnings("ignore")

# Path
import os


#### Lectura de dataset

Lectura del dataset de los datos del tiempo con la ayuda de la librería OS para poder trabajar sin problemas con las rutas del archivo.

In [50]:
PATH = os.getcwd()
DATA_PATH = os.path.join(PATH, 'data')

In [51]:
df: pd.DataFrame = pd.read_csv(os.path.join(DATA_PATH,'dxWeather.csv'))

# Pre - procesamiento de datos
- EDA.
- Transformación de datos.
- Visualización de datos.


Descripción del dataset:

- **Temperatura (numérica)**: La temperatura en grados Celsius, que varía desde frío extremo hasta calor extremo.  
- **Humedad (numérica)**: El porcentaje de humedad, incluyendo valores superiores al 100% para introducir valores atípicos.  
- **Velocidad del Viento (numérica)**: La velocidad del viento en kilómetros por hora, con un rango que incluye valores irrealistamente altos.  
- **Precipitación (%) (numérica)**: El porcentaje de precipitación, incluyendo valores atípicos.  
- **Cobertura de Nubes (categórica)**: La descripción de la cobertura de nubes.  
- **Presión Atmosférica (numérica)**: La presión atmosférica en hPa, cubriendo un amplio rango.  
- **Índice UV (numérica)**: El índice UV, que indica la intensidad de la radiación ultravioleta.  
- **Estación (categórica)**: La estación del año en la que se registraron los datos.  
- **Visibilidad (km) (numérica)**: La visibilidad en kilómetros, incluyendo valores muy bajos o muy altos.  
- **Ubicación (categórica)**: El tipo de ubicación donde se registraron los datos.  
- **Tipo de Clima (categórica)**: La variable objetivo para la clasificación, que indica el tipo de clima.  


[Link al dataset](https://www.kaggle.com/datasets/nikhil7280/weather-type-classification/) (1)

(1) es una modificacion del dataset original.



Estamos frente a un dataset que tiene 10.090 entradas y 8 columnas con tipos de datos int, float y object.

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10090 entries, 0 to 10089
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Temperatura         10090 non-null  int64  
 1   Humedad             10090 non-null  int64  
 2   VientoVelocidad     10090 non-null  float64
 3   Precipitation       10090 non-null  int64  
 4   PresionAtmosferica  10090 non-null  float64
 5   Localizacion        10090 non-null  object 
 6   TipoClima           10090 non-null  object 
 7   Estacion            10090 non-null  object 
dtypes: float64(2), int64(3), object(3)
memory usage: 630.8+ KB


La columna 'Precipitation' está en inglés, mientras que las demás están en español.

In [53]:
df.columns

Index(['Temperatura', 'Humedad', 'VientoVelocidad', 'Precipitation',
       'PresionAtmosferica', 'Localizacion', 'TipoClima', 'Estacion'],
      dtype='object')

Creo una copia para trabajar el dataset sin sobreescribir el original.

In [54]:
df_renamed: pd.DataFrame = df.copy()

In [55]:
df_renamed['Precipitacion'] = df_renamed['Precipitation']
df_renamed = df_renamed.drop(columns=['Precipitation'])

In [56]:
df_renamed.columns

Index(['Temperatura', 'Humedad', 'VientoVelocidad', 'PresionAtmosferica',
       'Localizacion', 'TipoClima', 'Estacion', 'Precipitacion'],
      dtype='object')

Exploración de una muestra chica del dataset.  

In [57]:
df_renamed.sample(5)

,Temperatura,Humedad,VientoVelocidad,PresionAtmosferica,Localizacion,TipoClima,Estacion,Precipitacion
4244,26,83,9.5,1018.87,Montania,Lluvioso,Verano,54
205,30,96,4.5,1001.66,Montania,Nublado,Verano,97
2638,30,71,9.5,1003.98,Montania,Nublado,Invierno,45
2066,32,48,4.0,1021.88,Llanura,Soleado,Otonio,7
9432,23,26,8.5,1026.74,Costa,Soleado,Otonio,9


Chequeo valores únicos en las columnas categóricas. De esta manera descarto la existencia de la letra 'ñ' en el caso de Otonio y Montania y que todos los campos incian con mayúsculas.

In [58]:
print(pd.unique(df_renamed['Estacion']))
print(pd.unique(df_renamed['Localizacion']))
print(pd.unique(df_renamed['TipoClima']))

['Primavera' 'Verano' 'Invierno' 'Otonio']
['Llanura' 'Montania' 'Costa']
['Nublado' 'Soleado' 'Nevado' 'Lluvioso']


Chequeo de valores faltantes: no existen faltantes en el dataset.

In [59]:
df_renamed.isna().sum()

,0
Temperatura,0
Humedad,0
VientoVelocidad,0
PresionAtmosferica,0
Localizacion,0
TipoClima,0
Estacion,0
Precipitacion,0


In [60]:
columnas_categoricas: list[str] = ['Localizacion', 'TipoClima', 'Estacion']
columnas_numericas: list[str] = ['Temperatura', 'Humedad', 'VientoVelocidad','PresionAtmosferica','Precipitacion']

Vemos que tenemos un gran desvío estandar en las variables Precipitacion, Temperatura y PresionAtmosferica

In [61]:
df_renamed.describe()

,Temperatura,Humedad,VientoVelocidad,PresionAtmosferica,Precipitacion
count,10090.000000,10090.000000,10090.000000,10090.000000,10090.000000
mean,21.872349,67.195045,9.634936,1007.685836,50.796333
std,16.426722,20.463797,6.795890,38.763971,32.333798
min,-24.000000,20.000000,0.000000,800.230000,0.000000
25%,13.000000,55.000000,5.000000,998.570000,18.000000
50%,23.000000,69.000000,8.500000,1010.315000,53.000000
75%,32.000000,81.000000,13.500000,1017.820000,80.000000
max,109.000000,109.000000,47.500000,1199.210000,109.000000


Debido a las distintas escalas de los datos, más allá que vemos necesario normalizar, se procede a visualizar los datos con un histograma.

In [62]:
df_renamed.columns

Index(['Temperatura', 'Humedad', 'VientoVelocidad', 'PresionAtmosferica',
       'Localizacion', 'TipoClima', 'Estacion', 'Precipitacion'],
      dtype='object')

In [63]:
histogram_ = px.histogram(df_renamed.drop(columns=['Estacion','TipoClima','Localizacion']), title='Histograma de los datos Temperatura, Humedad, Velocidad de viento y Presion Atmosferica', nbins=500)
histogram_.show()

Distribuciones observadas:
- Temperatura, Humedad, VientoVelocidad PresionAtmosferica tienen una distribucion normal. PresionAtmosferica tiene una colas pesadas tanto hacia la derecha como hacia la izquierda. VientoVelocidad presenta una cola pesada hacia la derecha.
- Precipitacion tiene a ser uniforme.

Análisis de outliers



In [64]:
px.box(df_renamed[columnas_numericas], width=1000, height=2000)

In [65]:
px.scatter(df_renamed['Temperatura'], color=df_renamed['Estacion'], title='Temperatura diferenciada por estación')

In [66]:
px.scatter(df_renamed['VientoVelocidad'], color=df_renamed['Estacion'], title='VientoVelocidad en cada Estacion')

En ambos scatterplots, podemos observar que las temperaturas no tienen relacion con las estaciones del anio. Esto se debe a que es un dataset generado de forma sintetica. En el caso del viento, se visualiza algo similar.

Procedemos a realizar la matriz de correlación para ver cual es la relación entre las variables.

In [67]:
corr = df_renamed[columnas_numericas].corr()

px.imshow(corr,  zmin=-1, zmax=1, title='Correlación de las variables', text_auto=True, color_continuous_scale='RdBu', aspect='auto')

No vemos correlaciones fuertes. Sin embargo, podemos destacar humedad - precipitacion. Esto es logico.

Teniendo en cuenta los análisis previos, se toma como criterio que los valores fuera del rango definido por
${Q1−1.5×IQR}$  y ${Q3+1.5×IQR}$ son considerados outliers. Este enfoque es robusto. Dado que muchas de las distribuciones son aproximadamente normales, optamos por rellenar esos outliers con la mediana de la columna correspondiente.

In [68]:
df_outliers: pd.DataFrame = pd.DataFrame()
df_without_outliers: pd.DataFrame = df_renamed.copy()

for column in columnas_numericas:
    q1 = df_renamed[column].quantile(0.25)
    q3 = df_renamed[column].quantile(0.75)
    iqr: float = q3 - q1

    lower_limit: float = q1 - 1.5 * iqr
    upper_limit: float = q3 + 1.5 * iqr


    median_ = df_renamed[column].median()

    outliers = df_renamed.loc[
        (df_renamed[column] < lower_limit) | (df_renamed[column] > upper_limit)
    ]

    df_outliers = pd.concat([df_outliers, outliers])
    df_without_outliers[column] = df_without_outliers[column].apply(
        lambda x: median_ if x < lower_limit or x > upper_limit else x
    )


df_outliers = df_outliers.drop_duplicates()

print(f'Existían {df_outliers.shape[0]} filas con outliers.')
print(f'Los outliers representaban un {100*df_outliers.shape[0]/df_renamed.shape[0]:.2f}% de la población.')


Existían 1311 filas con outliers.
Los outliers representaban un 12.99% de la población.


## Estandarización

Se utiliza la técnica Z-Score.

In [69]:
scaler: StandardScaler = StandardScaler()

df_scaled: pd.DataFrame = pd.DataFrame(scaler.fit_transform(df_without_outliers.drop(columns=columnas_categoricas)), columns=columnas_numericas)

In [70]:
df_scaled = pd.concat([df_scaled, df_without_outliers[columnas_categoricas]], axis=1)
df_scaled.head()

,Temperatura,Humedad,VientoVelocidad,PresionAtmosferica,Precipitacion,Localizacion,TipoClima,Estacion
0,1.234983,1.407675,-0.078074,0.248446,0.624878,Llanura,Nublado,Primavera
1,0.604399,-0.156139,-0.356764,0.879463,-1.076213,Montania,Soleado,Primavera
2,1.164918,0.772376,-1.378629,1.531255,0.965096,Costa,Soleado,Primavera
3,0.744529,-0.595962,-1.007041,0.127263,-0.766924,Llanura,Nublado,Verano
4,-1.287355,0.870114,-0.542558,-2.086060,1.398101,Llanura,Nevado,Invierno


Chequeamos que la estandarización de los datos es correcta.

In [71]:
df_scaled.describe()

,Temperatura,Humedad,VientoVelocidad,PresionAtmosferica,Precipitacion
count,1.009000e+04,1.009000e+04,1.009000e+04,1.009000e+04,1.009000e+04
mean,-4.982250e-17,2.042194e-16,-1.605587e-16,-6.823745e-16,-3.661865e-17
std,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00
min,-2.548524e+00,-2.306384e+00,-1.657319e+00,-3.360213e+00,-1.571076e+00
25%,-5.867054e-01,-5.959622e-01,-7.283512e-01,-6.454959e-01,-1.014355e+00
50%,1.139441e-01,8.820674e-02,-7.807379e-02,1.519321e-01,6.815705e-02
75%,6.744637e-01,6.746372e-01,6.651003e-01,7.000691e-01,9.032380e-01
max,2.706347e+00,2.042975e+00,3.173313e+00,3.252921e+00,1.800177e+00




Chequeo de balanceo de clases target: vemos que las clases están prácticamente balanceadas.


In [72]:
df_scaled['Estacion'].value_counts()

,count
Estacion,
Primavera,2598
Invierno,2500
Otonio,2500
Verano,2492


In [73]:
df_dummies: pd.DataFrame = pd.get_dummies(df_scaled, columns=['Localizacion', 'TipoClima'], dtype=int)
df_dummies

,Temperatura,Humedad,VientoVelocidad,PresionAtmosferica,Precipitacion,Estacion,Localizacion_Costa,Localizacion_Llanura,Localizacion_Montania,TipoClima_Lluvioso,TipoClima_Nevado,TipoClima_Nublado,TipoClima_Soleado
0,1.234983,1.407675,-0.078074,0.248446,0.624878,Primavera,0,1,0,0,0,1,0
1,0.604399,-0.156139,-0.356764,0.879463,-1.076213,Primavera,0,0,1,0,0,0,1
2,1.164918,0.772376,-1.378629,1.531255,0.965096,Primavera,1,0,0,0,0,0,1
3,0.744529,-0.595962,-1.007041,0.127263,-0.766924,Verano,0,1,0,0,0,1,0
4,-1.287355,0.870114,-0.542558,-2.086060,1.398101,Invierno,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10085,0.674464,-2.110908,-0.170971,1.822095,-1.416431,Verano,0,1,0,0,0,0,1
10086,0.534334,-0.253878,0.757997,-0.497696,-1.045284,Primavera,1,0,0,0,0,1,0
10087,-0.796900,0.332553,1.036687,-0.468266,0.624878,Verano,0,0,1,1,0,0,0
10088,0.604399,0.479160,-0.635454,0.357511,-0.705066,Otonio,1,0,0,0,0,1,0


## Implementacion de Modelos

### Separación de dataset en tran y test




Primeramente, se selecciona la variable target 'Estacion' y se crea también el dataset X con las variables restantes.


In [74]:
X: pd.DataFrame = df_dummies.drop(columns=['Estacion'])
y: pd.DataFrame = df_dummies[['Estacion']]



Separación en Train y Test. Se utiliza un 20% de para los datos de test y un 80% para los de train.


In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### SVC: Maquina de Vectores Soporte.

#### Linear Kernel



Definición de un diccionario que contiene los posibles valores de C (costo).


In [76]:
parametros_linear_kernal: Dict[str, List[float]] = {
    'C' : [0.1,0.05 ,0.2 ,0.5 ,0.8, 1 , 5, 10 , 40]
}

Se inicializa el modelo de SVM y se busca el mejor mediante GridSearch. Se utiliza 'recall' como métrica ya que es importante que diferencie bien entre categorías. No optimizamos gamma ya que no participa en el kernel lineal.

[Link Documentación de SVM, inciso 1.4.6](https://scikit-learn.org/stable/modules/svm.html)

In [77]:
recall_parameter: dict[float, float] = {}

for c in parametros_linear_kernal['C']:
    svc_linear = SVC(C=c, kernel='linear', random_state=42)
    svc_linear.fit(X_train, y_train)
    y_pred_svc_linear = svc_linear.predict(X_test
                                           )
    recall_parameter[c] = recall_score(y_test, y_pred_svc_linear, average='weighted')

    print(f'C={c}: \n')
    print(classification_report(y_test, y_pred_svc_linear))
    print("-------------")


C=0.1: 

              precision    recall  f1-score   support

    Invierno       0.86      0.54      0.67       498
      Otonio       0.26      0.20      0.23       506
   Primavera       0.30      0.51      0.38       526
      Verano       0.29      0.26      0.27       488

    accuracy                           0.38      2018
   macro avg       0.43      0.38      0.39      2018
weighted avg       0.43      0.38      0.39      2018

-------------
C=0.05: 

              precision    recall  f1-score   support

    Invierno       0.86      0.54      0.67       498
      Otonio       0.26      0.18      0.21       506
   Primavera       0.31      0.54      0.39       526
      Verano       0.29      0.25      0.26       488

    accuracy                           0.38      2018
   macro avg       0.43      0.38      0.38      2018
weighted avg       0.43      0.38      0.38      2018

-------------
C=0.2: 

              precision    recall  f1-score   support

    Invierno       

Se observa que a partir de un C = 0.2, se mantienen las métricas. Por ende, elegimos ese.

Cross - validation:

In [78]:
svc_linear_kfold: SVC = SVC(C=0.2, kernel='linear', random_state=42)

In [79]:
sfolder: StratifiedKFold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

recall_metrics: dict[str, float] = {}
f1_metrics: dict[str, float] = {}
precision_metrics: dict[str, float] = {}
accuracy_metrics: dict[str, float] = {}

gen: Generator = sfolder.split(X, y)
count: int = 1
for train_index, test_index in gen:

    x_train_fold, x_test_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

    svc_linear_kfold.fit(x_train_fold, y_train_fold)

    y_pred_svc_linear_kfold = svc_linear_kfold.predict(x_test_fold)

    recall_metrics[f'Recall {count}'] = recall_score(y_test_fold, y_pred_svc_linear_kfold, average='macro')
    f1_metrics[f'F1 - {count}'] = f1_score(y_test_fold, y_pred_svc_linear_kfold, average='macro')
    precision_metrics[f'Precision {count}'] = precision_score(y_test_fold, y_pred_svc_linear_kfold, average='macro')
    accuracy_metrics[f'Accuracy {count}'] = accuracy_score(y_test_fold, y_pred_svc_linear_kfold)

    print(f'Métricas del conjunto nro: {count}')
    print(f'Recall: {recall_metrics[f"Recall {count}"]}')
    print(f'F1: {f1_metrics[f"F1 - {count}"]}')
    print(f'Precision: {precision_metrics[f"Precision {count}"]}')
    print(f'Accuracy: {accuracy_metrics[f"Accuracy {count}"]}')
    print("--------------------------------------------------------------------------------")

    count += 1

Métricas del conjunto nro: 1
Recall: 0.376428405931418
F1: 0.3679843485985434
Precision: 0.4266555508957243
Accuracy: 0.37908820614469774
--------------------------------------------------------------------------------
Métricas del conjunto nro: 2
Recall: 0.3618456904541242
F1: 0.37655864478175116
Precision: 0.4181100027530443
Accuracy: 0.3622398414271556
--------------------------------------------------------------------------------
Métricas del conjunto nro: 3
Recall: 0.3700940685820204
F1: 0.36624127516017496
Precision: 0.42646708568879593
Accuracy: 0.3726461843409316
--------------------------------------------------------------------------------
Métricas del conjunto nro: 4
Recall: 0.37779792919171673
F1: 0.34158404556182737
Precision: 0.35603311679616106
Accuracy: 0.38057482656095143
--------------------------------------------------------------------------------
Métricas del conjunto nro: 5
Recall: 0.3823974268382623
F1: 0.39684925315984443
Precision: 0.43500419492236486
Accura

#### Gaussian Kernel

Generamos un modelo sin optimizar parámetros, por default para tener una base de comparación.

In [80]:
svc_gaussian_kernel_base: SVC = SVC(kernel='rbf', random_state=42).fit(X_train, y_train)

y_pred_svc_gaussian_kernel_base = svc_gaussian_kernel_base.predict(X_test)

print(classification_report(y_test, y_pred_svc_gaussian_kernel_base))

              precision    recall  f1-score   support

    Invierno       1.00      0.53      0.69       498
      Otonio       0.27      0.26      0.27       506
   Primavera       0.31      0.44      0.37       526
      Verano       0.28      0.31      0.29       488

    accuracy                           0.38      2018
   macro avg       0.47      0.38      0.40      2018
weighted avg       0.46      0.38      0.40      2018



En general, vemos una performance muy similar al kernel lineal optimizado.

#### Optimización

In [81]:
parameters_gaussian_kernel: dict[str, list[float]] = {
    'gamma' : [0.001, 0.01, 0.1, 1, 10, 40 ],
    'C' : [0.001, 0.01, 0.1, 1, 10, 40, 'scale', 'auto']
}

grid_search_gaussian_kernel = GridSearchCV(svc_gaussian_kernel_base, parameters_gaussian_kernel, scoring='accuracy', verbose=3, cv=2)

grid_search_gaussian_kernel.fit(X_train, y_train)

grid_search_gaussian_kernel.best_estimator_

Fitting 2 folds for each of 48 candidates, totalling 96 fits
[CV 1/2] END ..............C=0.001, gamma=0.001;, score=0.257 total time=   2.2s
[CV 2/2] END ..............C=0.001, gamma=0.001;, score=0.257 total time=   3.7s
[CV 1/2] END ...............C=0.001, gamma=0.01;, score=0.257 total time=   2.2s
[CV 2/2] END ...............C=0.001, gamma=0.01;, score=0.257 total time=   2.2s
[CV 1/2] END ................C=0.001, gamma=0.1;, score=0.257 total time=   2.1s
[CV 2/2] END ................C=0.001, gamma=0.1;, score=0.257 total time=   2.1s
[CV 1/2] END ..................C=0.001, gamma=1;, score=0.257 total time=   2.6s
[CV 2/2] END ..................C=0.001, gamma=1;, score=0.257 total time=   3.2s
[CV 1/2] END .................C=0.001, gamma=10;, score=0.257 total time=   2.2s
[CV 2/2] END .................C=0.001, gamma=10;, score=0.257 total time=   2.2s
[CV 1/2] END .................C=0.001, gamma=40;, score=0.257 total time=   2.9s
[CV 2/2] END .................C=0.001, gamma=40;

SVC(C=0.01, gamma=0.1, random_state=42)

In [82]:
y_pred_svc_gaussian_kernel_grid_search = grid_search_gaussian_kernel.predict(X_test)

print(classification_report(y_test, y_pred_svc_gaussian_kernel_grid_search))

              precision    recall  f1-score   support

    Invierno       0.96      0.53      0.68       498
      Otonio       0.00      0.00      0.00       506
   Primavera       0.30      0.99      0.46       526
      Verano       0.00      0.00      0.00       488

    accuracy                           0.39      2018
   macro avg       0.31      0.38      0.29      2018
weighted avg       0.31      0.39      0.29      2018



In [83]:
svc_gaussian_kfold: SVC = SVC(kernel='rbf', random_state=42).fit(X_train, y_train)
sfolder_gaussian: StratifiedKFold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

recall_metrics_gaussian: dict[str, float] = {}
f1_metrics_gaussian: dict[str, float] = {}
precision_metrics_gaussian: dict[str, float] = {}
accuracy_metrics_gaussian: dict[str, float] = {}

gen_gaussian: Generator = sfolder_gaussian.split(X, y)
count_gaussian: int = 1

for train_index, test_index in gen_gaussian:

    x_train_fold_gaussian, x_test_fold_gaussian = X.iloc[train_index], X.iloc[test_index]
    y_train_fold_gaussian, y_test_fold_gaussian = y.iloc[train_index], y.iloc[test_index]

    svc_gaussian_kfold.fit(x_train_fold_gaussian, y_train_fold_gaussian)

    y_pred_svc_gaussian_kfold = svc_gaussian_kernel_base.predict(x_test_fold_gaussian)

    recall_metrics_gaussian[f'Recall {count_gaussian}'] = recall_score(y_test_fold_gaussian, y_pred_svc_gaussian_kfold, average='macro')
    f1_metrics_gaussian[f'F1 - {count_gaussian}'] = f1_score(y_test_fold_gaussian, y_pred_svc_gaussian_kfold, average='macro')
    precision_metrics_gaussian[f'Precision {count_gaussian}'] = precision_score(y_test_fold_gaussian, y_pred_svc_gaussian_kfold, average='macro')
    accuracy_metrics_gaussian[f'Accuracy {count_gaussian}'] = accuracy_score(y_test_fold_gaussian, y_pred_svc_gaussian_kfold)

    print(f'Métricas del conjunto nro: {count_gaussian}')
    print(f'Recall: {recall_metrics_gaussian[f"Recall {count_gaussian}"]}')
    print(f'F1: {f1_metrics_gaussian[f"F1 - {count_gaussian}"]}')
    print(f'Precision: {precision_metrics_gaussian[f"Precision {count_gaussian}"]}')
    print(f'Accuracy: {accuracy_metrics_gaussian[f"Accuracy {count_gaussian}"]}')
    print("--------------------------------------------------------------------------------")

    count_gaussian += 1

Métricas del conjunto nro: 1
Recall: 0.44491110596231076
F1: 0.46019360895612427
Precision: 0.5178276621143022
Accuracy: 0.44598612487611494
--------------------------------------------------------------------------------
Métricas del conjunto nro: 2
Recall: 0.43676212542477605
F1: 0.45336338278432553
Precision: 0.5122894440347606
Accuracy: 0.4375619425173439
--------------------------------------------------------------------------------
Métricas del conjunto nro: 3
Recall: 0.44353622181031815
F1: 0.46115183290463735
Precision: 0.5193197824381042
Accuracy: 0.44400396432111
--------------------------------------------------------------------------------
Métricas del conjunto nro: 4
Recall: 0.4309387792926894
F1: 0.448139877611164
Precision: 0.5002529162335219
Accuracy: 0.43161546085232905
--------------------------------------------------------------------------------
Métricas del conjunto nro: 5
Recall: 0.42681461960529926
F1: 0.4453479839874654
Precision: 0.5011065100921448
Accuracy:

## Random Forest

In [84]:
rf: RandomForestClassifier = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

    Invierno       0.82      0.55      0.66       498
      Otonio       0.24      0.27      0.25       506
   Primavera       0.28      0.29      0.28       526
      Verano       0.26      0.30      0.28       488

    accuracy                           0.35      2018
   macro avg       0.40      0.35      0.37      2018
weighted avg       0.40      0.35      0.37      2018



In [85]:
px.imshow(confusion_matrix(y_test, y_pred_rf), text_auto=True, title='Matriz de confusión Random Forest')

In [90]:
param_dist: dict[str,int] = {'n_estimators': randint(50,500), 'max_depth': randint(1,20)}

rand_search = RandomizedSearchCV(rf, param_distributions = param_dist, n_iter=5, cv=5)
rand_search.fit(X_train, y_train)
print('Best hyperparameters:', rand_search.best_params_)

Best hyperparameters: {'max_depth': 2, 'n_estimators': 485}


In [91]:
best_rf = rand_search.best_estimator_
y_pred_best_rf = best_rf.predict(X_test)

print(classification_report(y_test, y_pred_best_rf))

              precision    recall  f1-score   support

    Invierno       0.84      0.54      0.66       498
      Otonio       0.24      0.01      0.02       506
   Primavera       0.31      0.91      0.46       526
      Verano       0.29      0.06      0.10       488

    accuracy                           0.39      2018
   macro avg       0.42      0.38      0.31      2018
weighted avg       0.42      0.39      0.31      2018



In [92]:
px.imshow(confusion_matrix(y_test, y_pred_best_rf), text_auto=True, title='Matriz de confusión mejor Random Forest')

In [93]:
rf_kfold: RandomForestClassifier = RandomForestClassifier(random_state=42)
sfolder_rf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

recall_metrics_rf: dict[str, float] = {}
f1_metrics_rf: dict[str, float] = {}
precision_metrics_rf: dict[str, float] = {}
accuracy_metrics_rf: dict[str, float] = {}

gen_rf: Generator = sfolder_rf.split(X, y)
count_rf: int = 1

for train_index, test_index in gen_rf:
    x_train_fold_rf, x_test_fold_rf = X.iloc[train_index], X.iloc[test_index]
    y_train_fold_rf, y_test_fold_rf = y.iloc[train_index], y.iloc[test_index]

    # Perform RandomizedSearchCV to find the best hyperparameters
    rf_kfold.fit(x_train_fold_rf, y_train_fold_rf)

    # Print the best hyperparameters
    print(f'Best hyperparameters for fold {count_rf}:', rand_search.best_params_)

    # Predict using the best model
    y_pred_rf_kfold = best_rf.predict(x_test_fold_rf)

    recall_metrics_rf[f'Recall {count_rf}'] = recall_score(y_test_fold_rf, y_pred_best_rf, average='macro')
    f1_metrics_rf[f'F1 - {count_rf}'] = f1_score(y_test_fold_rf, y_pred_best_rf, average='macro')
    precision_metrics_rf[f'Precision {count_rf}'] = precision_score(y_test_fold_rf, y_pred_best_rf, average='macro')
    accuracy_metrics_rf[f'Accuracy {count_rf}'] = accuracy_score(y_test_fold_rf, y_pred_best_rf)

    print(f'Métricas del conjunto nro: {count_rf}')
    print(f'Recall: {recall_metrics_rf[f"Recall {count_rf}"]}')
    print(f'F1: {f1_metrics_rf[f"F1 - {count_rf}"]}')
    print(f'Precision: {precision_metrics_rf[f"Precision {count_rf}"]}')
    print(f'Accuracy: {accuracy_metrics_rf[f"Accuracy {count_rf}"]}')
    print("--------------------------------------------------------------------------------")

    count_rf += 1

Best hyperparameters for fold 1: {'max_depth': 2, 'n_estimators': 485}
Métricas del conjunto nro: 1
Recall: 0.2552232777262898
F1: 0.1776882631405253
Precision: 0.2754093398811372
Accuracy: 0.2606541129831516
--------------------------------------------------------------------------------
Best hyperparameters for fold 2: {'max_depth': 2, 'n_estimators': 485}
Métricas del conjunto nro: 2
Recall: 0.24869199876428794
F1: 0.16738406091143157
Precision: 0.23270644667633195
Accuracy: 0.25421209117938554
--------------------------------------------------------------------------------
Best hyperparameters for fold 3: {'max_depth': 2, 'n_estimators': 485}
Métricas del conjunto nro: 3
Recall: 0.26122930182267534
F1: 0.18528822575946075
Precision: 0.2784321658206298
Accuracy: 0.2666005946481665
--------------------------------------------------------------------------------
Best hyperparameters for fold 4: {'max_depth': 2, 'n_estimators': 485}
Métricas del conjunto nro: 4
Recall: 0.25275718875129